#사전 작업

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#데이터 전처리


In [99]:
#import 및 파일 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, classification_report
from scipy import stats
from sklearn.preprocessing import MinMaxScaler

plt.rc('font', family='NanumGothic')

file_path = "/content/drive/MyDrive/Colab Notebooks/01 DATA/accidentInfoList.CSV"
df = pd.read_csv(file_path, encoding='cp949')



In [100]:
# 숫자 추출 함수 생성
def extract_numbers_from_age(age):
    numbers = re.findall(r'\d+', str(age))
    if numbers:
        return int(numbers[0])
    else:
        return np.nan

# 문자 데이터 제거 및 숫자만 추출
df['가해운전자 연령'] = df['가해운전자 연령'].apply(extract_numbers_from_age)
df['피해운전자 연령'] = df['피해운전자 연령'].apply(extract_numbers_from_age)
mean_age_driver = round(df['가해운전자 연령'].mean(), 1)
mean_age_victim = round(df['피해운전자 연령'].mean(), 1)

# 결측치 처리
df['가해운전자 연령'].fillna(mean_age_driver, inplace=True)
df['피해운전자 연령'].fillna(mean_age_victim, inplace=True)
df['피해운전자 차종'].fillna('기타불명', inplace=True)
df['피해운전자 성별'].fillna('기타불명', inplace=True)


In [101]:
# 피쳐 생성
df['ECLO'] = df['사망자수'] * 10 + df['중상자수'] * 5 + df['경상자수'] * 3 + df['부상신고자수'] * 1
df.drop(columns=['사망자수','경상자수','중상자수','부상신고자수'], inplace=True)

# '사고일시' 열에서 시간 정보 추출
df['사고일시'] = pd.to_datetime(df['사고일시'], format='%Y년 %m월 %d일 %H시')
df['사고발생시간'] = df['사고일시'].dt.hour
df.drop(columns=['사고일시'], inplace=True)

# '시군구' 피처에서 '구' 정보만 남기기
df['시군구'] = df['시군구'].apply(lambda x: x.split(' ')[1])

#피쳐 제거
df.drop(columns=['사고번호','사고내용','시군구','가해운전자 상해정도','피해운전자 상해정도'], inplace=True)

In [102]:
#범주형 데이터 처리
# '요일'
def categorize_weekday(weekday):
    if weekday in ['월요일', '화요일', '수요일', '목요일', '금요일']:
        return '평일'
    else:
        return '주말'

df['평일/주말'] = df['요일'].apply(categorize_weekday)
df.drop(columns=['요일'], inplace=True)

# 사고발생시간
def recategorize_time_of_day(hour):
    if 21 <= hour <= 24 or 0 <= hour < 6:
        return '야간'
    else:
        return '주간'

df['시간대'] = df['사고발생시간'].apply(recategorize_time_of_day)
df.drop(columns=['사고발생시간'], inplace=True)

# 연령
def categorize_age(age):
    if age <= 19:
        return '미성년층'
    elif 20 <= age <= 45:
        return '청년층'
    elif 46 <= age <= 60:
        return '중장년층'
    else:
        return '노년층'

df['가해운전자 연령대'] = df['가해운전자 연령'].apply(categorize_age)
df.drop(columns=['가해운전자 연령'], inplace=True)
df['피해운전자 연령대'] = df['피해운전자 연령'].apply(categorize_age)
df.drop(columns=['피해운전자 연령'], inplace=True)


In [103]:
#인코딩
def one_hot_encode_features(df, categorical_features):
    for feature in categorical_features:
        encoded_df = pd.get_dummies(df[feature], prefix=feature)
        df = df.drop(columns=[feature])
        df = pd.concat([df, encoded_df], axis=1)

    return df

categorical_features = ['사고유형', '법규위반', '노면상태', '기상상태', '도로형태',
                        '가해운전자 차종', '가해운전자 성별', '피해운전자 차종',
                        '피해운전자 성별', '평일/주말', '시간대',
                        '가해운전자 연령대', '피해운전자 연령대']

df = one_hot_encode_features(df, categorical_features)


In [104]:
# 이상치 제거
z_scores = np.abs(stats.zscore(df['ECLO']))
outliers = (z_scores > 3)
df_no_outliers = df[~outliers]


In [ ]:
# MINMAX 스케일러 적용
minmax_scaler = MinMaxScaler()
eclo_minmax_scaled = minmax_scaler.fit_transform(eclo_data)
df_no_outliers['ECLO_scaled'] = eclo_minmax_scaled


#모델링

In [106]:
# 피처 선택
features = ['사고유형_차대사람 - 기타', '사고유형_차대사람 - 길가장자리구역통행중', '사고유형_차대사람 - 보도통행중', '사고유형_차대사람 - 차도통행중', '사고유형_차대사람 - 횡단중', '사고유형_차대차 - 기타', '사고유형_차대차 - 정면충돌', '사고유형_차대차 - 추돌', '사고유형_차대차 - 측면충돌', '사고유형_차대차 - 후진중충돌', '사고유형_차량단독 - 공작물충돌', '사고유형_차량단독 - 기타', '사고유형_차량단독 - 도로외이탈 - 기타', '사고유형_차량단독 - 도로외이탈 - 추락', '사고유형_차량단독 - 전도전복 - 전도', '사고유형_차량단독 - 전도전복 - 전복', '사고유형_차량단독 - 주/정차차량 충돌', '법규위반_과속', '법규위반_교차로운행방법위반', '법규위반_기타', '법규위반_보행자보호의무위반', '법규위반_불법유턴', '법규위반_신호위반', '법규위반_안전거리미확보', '법규위반_안전운전불이행', '법규위반_중앙선침범', '법규위반_직진우회전진행방해', '법규위반_차로위반', '노면상태_건조', '노면상태_기타', '노면상태_서리/결빙', '노면상태_적설', '노면상태_젖음/습기', '노면상태_침수', '노면상태_해빙', '기상상태_기타', '기상상태_눈', '기상상태_맑음', '기상상태_비', '기상상태_안개', '기상상태_흐림', '도로형태_교차로 - 교차로부근', '도로형태_교차로 - 교차로안', '도로형태_교차로 - 교차로횡단보도내', '도로형태_기타 - 기타', '도로형태_단일로 - 고가도로위', '도로형태_단일로 - 교량', '도로형태_단일로 - 기타', '도로형태_단일로 - 지하차도(도로)내', '도로형태_단일로 - 터널', '도로형태_미분류 - 미분류', '도로형태_주차장 - 주차장', '가해운전자 차종_개인형이동수단(PM)', '가해운전자 차종_건설기계', '가해운전자 차종_기타불명', '가해운전자 차종_농기계', '가해운전자 차종_사륜오토바이(ATV)', '가해운전자 차종_승용', '가해운전자 차종_승합', '가해운전자 차종_원동기', '가해운전자 차종_이륜', '가해운전자 차종_자전거', '가해운전자 차종_특수', '가해운전자 차종_화물', '가해운전자 성별_기타불명', '가해운전자 성별_남', '가해운전자 성별_여', '피해운전자 차종_개인형이동수단(PM)', '피해운전자 차종_건설기계', '피해운전자 차종_기타불명', '피해운전자 차종_보행자', '피해운전자 차종_사륜오토바이(ATV)', '피해운전자 차종_승용', '피해운전자 차종_승합', '피해운전자 차종_원동기', '피해운전자 차종_이륜', '피해운전자 차종_자전거', '피해운전자 차종_특수', '피해운전자 차종_화물', '피해운전자 성별_기타불명', '피해운전자 성별_남', '피해운전자 성별_여', '평일/주말_주말', '평일/주말_평일', '시간대_야간', '시간대_주간', '가해운전자 연령대_노년층', '가해운전자 연령대_미성년층', '가해운전자 연령대_중장년층', '가해운전자 연령대_청년층', '피해운전자 연령대_노년층', '피해운전자 연령대_미성년층', '피해운전자 연령대_중장년층', '피해운전자 연령대_청년층']


In [107]:
#회귀 모델 예시

# 독립변수와 종속변수 분할
X = df_no_outliers[features]
y = df_no_outliers['ECLO_scaled']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#모델 훈련
rf_regressor = RandomForestRegressor(random_state=42)
rf_regressor.fit(X_train, y_train)

# 예측
y_pred_train_rf = rf_regressor.predict(X_train)
y_pred_test_rf = rf_regressor.predict(X_test)

# 성능 평가
print("Random Forest Regressor 성능 평가:")
print("훈련 세트 R2 점수:", r2_score(y_train, y_pred_train_rf))
print("테스트 세트 R2 점수:", r2_score(y_test, y_pred_test_rf))
print("훈련 세트 RMSE:", mean_squared_error(y_train, y_pred_train_rf, squared=False))
print("테스트 세트 RMSE:", mean_squared_error(y_test, y_pred_test_rf, squared=False))
print("훈련 세트 MAE:", mean_absolute_error(y_train, y_pred_train_rf))
print("테스트 세트 MAE:", mean_absolute_error(y_test, y_pred_test_rf))

# 피처 중요도 출력
print("\nRandom Forest Regressor 피처 중요도:")
rf_feature_importances = rf_regressor.feature_importances_
for i, feature in enumerate(features):
    print(f"{feature}: {rf_feature_importances[i]}")


Random Forest Regressor 성능 평가:
훈련 세트 R2 점수: 0.49538691321927586
테스트 세트 R2 점수: -0.07156369526844997
훈련 세트 RMSE: 0.12655853871465844
테스트 세트 RMSE: 0.18289871863853802
훈련 세트 MAE: 0.08824363969148007
테스트 세트 MAE: 0.13700896075193073

Random Forest Regressor 피처 중요도:
사고유형_차대사람 - 기타: 0.004247259713282374
사고유형_차대사람 - 길가장자리구역통행중: 0.0016082778053269182
사고유형_차대사람 - 보도통행중: 0.002616451418736838
사고유형_차대사람 - 차도통행중: 0.0030324221704515044
사고유형_차대사람 - 횡단중: 0.004307768357853614
사고유형_차대차 - 기타: 0.020953059715309294
사고유형_차대차 - 정면충돌: 0.009950408973708564
사고유형_차대차 - 추돌: 0.014751739013189385
사고유형_차대차 - 측면충돌: 0.022366623121863306
사고유형_차대차 - 후진중충돌: 0.004372960177508877
사고유형_차량단독 - 공작물충돌: 0.0011269658583865727
사고유형_차량단독 - 기타: 0.0010133987657594634
사고유형_차량단독 - 도로외이탈 - 기타: 2.427879938165764e-05
사고유형_차량단독 - 도로외이탈 - 추락: 0.00022096769429248166
사고유형_차량단독 - 전도전복 - 전도: 0.0006133229878646112
사고유형_차량단독 - 전도전복 - 전복: 0.00025835744666462913
사고유형_차량단독 - 주/정차차량 충돌: 2.702272410917235e-05
법규위반_과속: 0.002391745879157578
법규위반_교차로운행방법위

In [108]:
#분류형 모델 예시(ECLO 값 분포가 넓지 않아서 따로 범주화 진행하지는 않음)

# 독립변수와 종속변수 분할
X = df_no_outliers[features]
y = df_no_outliers['ECLO']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# RandomForestClassifier 모델 훈련
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

# 예측
y_pred_train_rf_clf = rf_classifier.predict(X_train)
y_pred_test_rf_clf = rf_classifier.predict(X_test)

# 성능 평가
print("Random Forest Classifier 성능 평가:")
print("훈련 정확도:", accuracy_score(y_train, y_pred_train_rf_clf))
print("테스트 정확도:", accuracy_score(y_test, y_pred_test_rf_clf))

# 피처 중요도 출력
print("\nRandom Forest Classifier 피처 중요도:")
rf_feature_importances = rf_classifier.feature_importances_
for i, feature in enumerate(features):
    print(f"{feature}: {rf_feature_importances[i]}")


Random Forest Classifier 성능 평가:
훈련 정확도: 0.7659390800345295
테스트 정확도: 0.49716371528634146

Random Forest Classifier 피처 중요도:
사고유형_차대사람 - 기타: 0.006432494525483153
사고유형_차대사람 - 길가장자리구역통행중: 0.002510009434299779
사고유형_차대사람 - 보도통행중: 0.00334431990485517
사고유형_차대사람 - 차도통행중: 0.003561414508116238
사고유형_차대사람 - 횡단중: 0.007079808015120678
사고유형_차대차 - 기타: 0.021590181607321263
사고유형_차대차 - 정면충돌: 0.008587062802541076
사고유형_차대차 - 추돌: 0.01419022453548966
사고유형_차대차 - 측면충돌: 0.022310436313331388
사고유형_차대차 - 후진중충돌: 0.0036923106658850453
사고유형_차량단독 - 공작물충돌: 0.000879396761120392
사고유형_차량단독 - 기타: 0.0011353508300385856
사고유형_차량단독 - 도로외이탈 - 기타: 4.2209482360359936e-05
사고유형_차량단독 - 도로외이탈 - 추락: 8.993574713793976e-05
사고유형_차량단독 - 전도전복 - 전도: 0.000702464687293464
사고유형_차량단독 - 전도전복 - 전복: 0.00014682554372915792
사고유형_차량단독 - 주/정차차량 충돌: 3.63023565079022e-05
법규위반_과속: 0.001087196124805442
법규위반_교차로운행방법위반: 0.010600480232504957
법규위반_기타: 0.011235988719273692
법규위반_보행자보호의무위반: 0.004447438369399715
법규위반_불법유턴: 0.0041129062714318956
법규위반_신호위반: 0.0167863

#SUB

In [ ]:
print(df.columns.tolist())

In [ ]:
#원-핫 인코딩 예시
categorical_features = ['요일']
encoded_df = pd.get_dummies(df[categorical_features])
df = df.drop(columns=categorical_features)
df = pd.concat([df, encoded_df], axis=1)
print(encoded_df.columns)


#시각화 관련

In [ ]:
!apt-get install -y fonts-nanum*
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

데이터탐색에 들어갈 자료 생성

In [ ]:
# 빈도표
plt.figure(figsize=(16, 12))
sns.countplot(x='피해운전자 연령', data=df, palette='viridis')
plt.ylabel('빈도')
plt.show()


In [ ]:
# 커널 밀도 추정 그래프
plt.figure(figsize=(10, 6))
sns.kdeplot(data=df, x='사고발생시간', fill=True, color='skyblue')
plt.title('사고발생시간별 분포')
plt.xlabel('사고발생시간')
plt.ylabel('밀도')
plt.show()


시각화에 들어갈 자료 생성

In [ ]:
# 사고내용별 ECLO
avg_eclo_by_accident = df.groupby('시군구')['ECLO'].mean().reset_index()
avg_eclo_all = df['ECLO'].mean()

# 막대 그래프
plt.figure(figsize=(12, 8))
sns.barplot(x='시군구', y='ECLO', data=avg_eclo_by_accident, palette='viridis')
plt.ylabel('평균 ECLO')
plt.axhline(avg_eclo_all, color='r', linestyle='--', label='전체 평균 ECLO')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
def categorize_age(age):
    if age <= 19:
        return '0~19'
    elif age <= 45:
        return '20~45'
    elif age <= 60:
        return '46~60'
    else:
        return '61 이상'

# 연령을 범주
df['피해운전자 연령대'] = df['피해운전자 연령'].apply(categorize_age)

# 사고내용별 ECLO
avg_eclo_by_accident = df.groupby('피해운전자 연령대')['ECLO'].mean().reset_index()
avg_eclo_all = df['ECLO'].mean()

# 막대 그래프
plt.figure(figsize=(12, 8))
sns.barplot(x='피해운전자 연령대', y='ECLO', data=avg_eclo_by_accident, palette='viridis')
plt.ylabel('평균 ECLO')
plt.axhline(avg_eclo_all, color='r', linestyle='--', label='전체 평균 ECLO')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 3가지 이상의 피처를 활용한 히트맵

heatmap_data = df.pivot_table(index='사고발생시간', columns='법규위반', values='ECLO', aggfunc='mean')

plt.figure(figsize=(14, 10))
sns.heatmap(heatmap_data, cmap='viridis', annot=True, fmt=".2f", linewidths=.5)
plt.title('사고 발생시간과 법규위반에 따른 ECLO 평균')
plt.xlabel('법규위반')
plt.ylabel('사고발생시간')
plt.show()
